In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow.compat.v1 as tf
import sys

sys.path.append(r"C:\Users\mohit\Desktop\data")

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

flags = tf.app.flags
flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
flags.DEFINE_string('image_dir', '', 'Path to images')
FLAGS = flags.FLAGS

c:\users\mohit\appdata\local\programs\python\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\mohit\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\mohit\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
def split(df, group):
    data = namedtuple("data", ["filename", "object"])
    gb = df.groupby(group)
    return [
        data(filename, gb.get_group(x))
        for filename, x in zip(gb.groups.keys(), gb.groups)
    ]


def create_tf_example(group, path, label_map):
    print(group)
    print(path)
    with tf.gfile.GFile(os.path.join(path, "{}".format(group.filename)), "rb") as fid:
        encoded_jpg = fid.read()
        
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    
    image = Image.open(encoded_jpg_io)
    width, height = image.size
    print("hi")

    filename = group.filename.encode("utf8")
    print(filename)
    image_format = b"jpg"
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row["xmin"] / width)
        xmaxs.append(row["xmax"] / width)
        ymins.append(row["ymin"] / height)
        ymaxs.append(row["ymax"] / height)
        classes_text.append(row["class"].encode("utf8"))
        class_index = label_map.get(row["class"])
        assert (
            class_index is not None
        ), "class label: `{}` not found in label_map: {}".format(
            row["class"], label_map
        )
        classes.append(class_index)

    tf_example = tf.train.Example(
        features=tf.train.Features(
            feature={
                "image/height": dataset_util.int64_feature(height),
                "image/width": dataset_util.int64_feature(width),
                "image/filename": dataset_util.bytes_feature(filename),
                "image/source_id": dataset_util.bytes_feature(filename),
                "image/encoded": dataset_util.bytes_feature(encoded_jpg),
                "image/format": dataset_util.bytes_feature(image_format),
                "image/object/bbox/xmin": dataset_util.float_list_feature(xmins),
                "image/object/bbox/xmax": dataset_util.float_list_feature(xmaxs),
                "image/object/bbox/ymin": dataset_util.float_list_feature(ymins),
                "image/object/bbox/ymax": dataset_util.float_list_feature(ymaxs),
                "image/object/class/text": dataset_util.bytes_list_feature(
                    classes_text
                ),
                "image/object/class/label": dataset_util.int64_list_feature(classes),
            }
        )
    )
    return tf_example




In [3]:
output_path=r"C:\Users\mohit\Desktop\data\final1.record"
img_path=r"C:\Users\mohit\Desktop\data\test"
csv_input=r"C:\Users\mohit\Desktop\data\data\test_labels.csv"
label_map=r"C:\Users\mohit\Desktop\data\label_map.txt"

In [4]:
writer = tf.compat.v1.python_io.TFRecordWriter(output_path)
path = os.path.join(os.getcwd(), img_path)
examples = pd.read_csv(csv_input)

# Load the `label_map` from pbtxt file.
from object_detection.utils import label_map_util

label_map = label_map_util.load_labelmap(label_map)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=90, use_display_name=True
)
category_index = label_map_util.create_category_index(categories)
label_map = {}
for k, v in category_index.items():
    label_map[v.get("name")] = v.get("id")

grouped = split(examples, "filename")
for group in grouped:
    tf_example = create_tf_example(group, path, label_map)
    writer.write(tf_example.SerializeToString())

writer.close()
output_path = os.path.join(os.getcwd(), output_path)
print("Successfully created the TFRecords: {}".format(output_path))

data(filename='complete_204.jpg', object=           filename  width  height   class  xmin  ymin  xmax  ymax
0  complete_204.jpg    512     512  person   150    33   370   482
1  complete_204.jpg    512     512  helmet   232    41   305    97
2  complete_204.jpg    512     512    face   232    94   305   145
3  complete_204.jpg    512     512  jacket   185   152   340   367)
C:\Users\mohit\Desktop\data\test
hi
b'complete_204.jpg'
data(filename='complete_205.jpg', object=            filename  width  height   class  xmin  ymin  xmax  ymax
4   complete_205.jpg    512     512  person    43   208   129   512
5   complete_205.jpg    512     512  person   115   206   161   468
6   complete_205.jpg    512     512  person   356   162   508   512
7   complete_205.jpg    512     512  person   210   202   255   362
8   complete_205.jpg    512     512  person   289   198   327   405
9   complete_205.jpg    512     512  helmet   399   168   463   229
10  complete_205.jpg    512     512    face   414 

data(filename='complete_225.jpg', object=             filename  width  height   class  xmin  ymin  xmax  ymax
196  complete_225.jpg    512     512  person     1    22   484   512
197  complete_225.jpg    512     512  helmet    98    26   484   158
198  complete_225.jpg    512     512    face   204   139   433   275
199  complete_225.jpg    512     512  jacket     1   282   451   512)
C:\Users\mohit\Desktop\data\test
hi
b'complete_225.jpg'
data(filename='complete_226.jpg', object=             filename  width  height   class  xmin  ymin  xmax  ymax
200  complete_226.jpg    512     512  person     6   113   217   512
201  complete_226.jpg    512     512  person   241     5   501   512
202  complete_226.jpg    512     512  helmet   307    10   420   101
203  complete_226.jpg    512     512  helmet    74   118   190   196
204  complete_226.jpg    512     512    face    98   190   173   283
205  complete_226.jpg    512     512    face   318    80   410   186)
C:\Users\mohit\Desktop\data\test